In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
import h3
import shapely
from geopandas import GeoDataFrame,GeoSeries
import folium
import matplotlib
%matplotlib inline

/Users/rakeenahmed/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Pedestrian Crossover

In [2]:
#read in data
pedestrian_crossover = gpd.read_file('Pedestrian Crossover.geojson')

In [3]:
#convert column names to lowercase
pedestrian_crossover.columns = [s.strip().lower() for s in pedestrian_crossover.columns]

In [4]:
#drop unneccesary columns
pedestrian_crossover = pedestrian_crossover.drop(columns=['px','midblock_route','side2_street','private_access','additional_info','objectid','geo_id','node_id','mi_prinx'])

In [5]:
#rename id column
pedestrian_crossover.rename(columns={'_id':'id'},inplace=True)

In [6]:
#check remaning columns
pedestrian_crossover.columns

Index(['id', 'main_street', 'side1_street', 'geometry'], dtype='object')

In [7]:
#check remainig dataframe
pedestrian_crossover.head()

,id,main_street,side1_street,geometry
0,1,BATHURST ST,COLLEGE ST,MULTIPOINT (-79.40820 43.65771)
1,2,BATHURST ST,SCADDING CRT,MULTIPOINT (-79.40560 43.65127)
2,3,ANGLESEY BLVD,HARTFIELD RD,MULTIPOINT (-79.52851 43.66460)
3,4,BAYVIEW AVE,PARKHURST BLVD,MULTIPOINT (-79.37634 43.70870)
4,5,ANNETTE ST,DURIE ST,MULTIPOINT (-79.48257 43.65949)


In [7]:
#set id as index
pedestrian_crossover.set_index('id',inplace=True)

In [8]:
#using the geopandas explode() function to convert the multiline geometry to single line point geometry
#pedestrian_crossover_exploded = pedestrian_crossover.explode()

C:\Users\arara\AppData\Local\Temp\ipykernel_44436\3836950908.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  pedestrian_crossover_exploded = pedestrian_crossover.explode()


In [9]:
#extract latitude and longitudes from single line geometry
pedestrian_crossover_exploded['latitude'] = pedestrian_crossover_exploded.apply(lambda x: x.geometry.y, axis=1)
pedestrian_crossover_exploded['longitude'] = pedestrian_crossover_exploded.apply(lambda x: x.geometry.x, axis=1)

In [10]:
#checking new columns in dataframe
#pedestrian_crossover_exploded.head()

,,id,main_street,side1_street,geometry,latitude,longitude
0,0,1,BATHURST ST,COLLEGE ST,POINT (-79.40820 43.65771),43.657710,-79.408200
1,0,2,BATHURST ST,SCADDING CRT,POINT (-79.40560 43.65127),43.651267,-79.405604
2,0,3,ANGLESEY BLVD,HARTFIELD RD,POINT (-79.52851 43.66460),43.664597,-79.528512
3,0,4,BAYVIEW AVE,PARKHURST BLVD,POINT (-79.37634 43.70870),43.708704,-79.376338
4,0,5,ANNETTE ST,DURIE ST,POINT (-79.48257 43.65949),43.659494,-79.482574


In [11]:
#combining latitude and longitudes to form coordinates column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded.apply(lambda x: (x.latitude, x.longitude), axis=1)

In [12]:
#check dataframe
#pedestrian_crossover_exploded.head()

,,id,main_street,side1_street,geometry,latitude,longitude,coordinates
0,0,1,BATHURST ST,COLLEGE ST,POINT (-79.40820 43.65771),43.657710,-79.408200,"(43.6577100019432, -79.40819999547631)"
1,0,2,BATHURST ST,SCADDING CRT,POINT (-79.40560 43.65127),43.651267,-79.405604,"(43.6512669957427, -79.4056040041994)"
2,0,3,ANGLESEY BLVD,HARTFIELD RD,POINT (-79.52851 43.66460),43.664597,-79.528512,"(43.66459700173547, -79.52851199795252)"
3,0,4,BAYVIEW AVE,PARKHURST BLVD,POINT (-79.37634 43.70870),43.708704,-79.376338,"(43.708703999703154, -79.3763379938911)"
4,0,5,ANNETTE ST,DURIE ST,POINT (-79.48257 43.65949),43.659494,-79.482574,"(43.65949400063825, -79.48257400128824)"


In [13]:
#create hex cells for pedestrian crossings

#create coordinate column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded['latitude'].astype('str') + ',' + pedestrian_crossover_exploded['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
#def get_hex_cell(coordinates):
    #a = float(coordinates.split(',')[0])
    #b = float(coordinates.split(',')[1])
    a = coordinates[0]
    b = coordinates[1]
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
#pedestrian_crossover_exploded['hex_cell'] = pedestrian_crossover_exploded['coordinates'].apply(get_hex_cell)

In [14]:
#check new hex cell column
#pedestrian_crossover_exploded.head()

,,id,main_street,side1_street,geometry,latitude,longitude,coordinates,hex_cell
0,0,1,BATHURST ST,COLLEGE ST,POINT (-79.40820 43.65771),43.657710,-79.408200,"(43.6577100019432, -79.40819999547631)",8a2b9bc7375ffff
1,0,2,BATHURST ST,SCADDING CRT,POINT (-79.40560 43.65127),43.651267,-79.405604,"(43.6512669957427, -79.4056040041994)",8a2b9bc55957fff
2,0,3,ANGLESEY BLVD,HARTFIELD RD,POINT (-79.52851 43.66460),43.664597,-79.528512,"(43.66459700173547, -79.52851199795252)",8a2b9bce0d67fff
3,0,4,BAYVIEW AVE,PARKHURST BLVD,POINT (-79.37634 43.70870),43.708704,-79.376338,"(43.708703999703154, -79.3763379938911)",8a2b9bc74137fff
4,0,5,ANNETTE ST,DURIE ST,POINT (-79.48257 43.65949),43.659494,-79.482574,"(43.65949400063825, -79.48257400128824)",8a2b9bc5648ffff


In [8]:
#add flag for if pedestrian crossover is present
pedestrian_crossover['pedestrian_crossover'] = 'Yes'

In [9]:
#add prefix before column names
pedestrian_crossover.columns = ['pedestrian_crossover_' + s if s != 'geometry' else s for s in pedestrian_crossover.columns]

In [10]:
#saving file as geojson
pedestrian_crossover.to_file('pedestrian_crossover_procesesd.geojson')

# Traffic Signals

In [11]:
#reading in data
traffic_signals = gpd.read_file('Traffic Signal.geojson')

In [12]:
#convert column headers to lowercase
traffic_signals.columns = [s.strip().lower() for s in traffic_signals.columns]

In [13]:
traffic_signals.head()

,_id,px,main_street,midblock_route,side1_street,side2_street,private_access,additional_info,activationdate,signalsystem,...,mi_prinx,bicycle_signal,ups,led_blankout_sign,lpi_north_implementation_date,lpi_south_implementation_date,lpi_east_implementation_date,lpi_west_implementation_date,lpi_comment,geometry
0,1,0002,JARVIS ST,None,FRONT ST E,LOWER JARVIS ST,None,None,1948-11-15 05:00:00,TransSuite,...,1,0.0,0.0,0.0,2023-04-06T04:00:00,2023-04-06T04:00:00,2023-04-06T04:00:00,2023-04-06T04:00:00,None,MULTIPOINT (-79.37146 43.64945)
1,2,0003,KING ST E,None,JARVIS ST,None,None,None,1950-08-23 04:00:00,TransSuite,...,2,0.0,0.0,1.0,,,2021-06-23T04:00:00,2021-06-23T04:00:00,None,MULTIPOINT (-79.37191 43.65046)
2,3,0004,JARVIS ST,None,ADELAIDE ST E,None,None,None,1958-09-12 04:00:00,TransSuite,...,3,0.0,0.0,0.0,,,,,None,MULTIPOINT (-79.37237 43.65153)
3,4,0005,JARVIS ST,None,RICHMOND ST E,None,None,None,1962-04-21 04:00:00,TransSuite,...,4,0.0,1.0,0.0,2020-10-08T04:00:00,2020-10-08T04:00:00,,,None,MULTIPOINT (-79.37283 43.65270)
4,5,0006,JARVIS ST,None,QUEEN ST E,None,None,None,1928-08-24 04:00:00,TransSuite,...,5,0.0,0.0,0.0,2021-03-03T05:00:00,2021-03-03T05:00:00,2021-03-03T05:00:00,2021-03-03T05:00:00,None,MULTIPOINT (-79.37325 43.65373)


In [14]:
#rename id column
traffic_signals.rename(columns={'_id':'id'},inplace=True)

In [15]:
traffic_signals.columns

Index(['id', 'px', 'main_street', 'midblock_route', 'side1_street',
       'side2_street', 'private_access', 'additional_info', 'activationdate',
       'signalsystem', 'non_system', 'control_mode', 'pedwalkspeed',
       'aps_operation', 'numberofapproaches', 'objectid', 'geo_id', 'node_id',
       'audiblepedsignal', 'transit_preempt', 'fire_preempt', 'rail_preempt',
       'mi_prinx', 'bicycle_signal', 'ups', 'led_blankout_sign',
       'lpi_north_implementation_date', 'lpi_south_implementation_date',
       'lpi_east_implementation_date', 'lpi_west_implementation_date',
       'lpi_comment', 'geometry'],
      dtype='object')

In [16]:
#drop unneccesary columns
traffic_signals = traffic_signals.drop(columns=['px','midblock_route','side2_street','private_access','additional_info','activationdate','non_system','control_mode','pedwalkspeed','aps_operation'])

In [17]:
#drop more columns
traffic_signals.drop(traffic_signals.columns[4:-1],inplace=True,axis=1)

In [18]:
#check remaining columns
traffic_signals.columns

Index(['id', 'main_street', 'side1_street', 'signalsystem', 'geometry'], dtype='object')

In [19]:
traffic_signals.head()

,id,main_street,side1_street,signalsystem,geometry
0,1,JARVIS ST,FRONT ST E,TransSuite,MULTIPOINT (-79.37146 43.64945)
1,2,KING ST E,JARVIS ST,TransSuite,MULTIPOINT (-79.37191 43.65046)
2,3,JARVIS ST,ADELAIDE ST E,TransSuite,MULTIPOINT (-79.37237 43.65153)
3,4,JARVIS ST,RICHMOND ST E,TransSuite,MULTIPOINT (-79.37283 43.65270)
4,5,JARVIS ST,QUEEN ST E,TransSuite,MULTIPOINT (-79.37325 43.65373)


In [20]:
#set id as index
traffic_signals.set_index('id',inplace=True)

In [64]:
#using the geopandas explode() function to convert the multiline geometry to single line point geometry
#traffic_signals_exploded = traffic_signals.explode()

C:\Users\arara\AppData\Local\Temp\ipykernel_44436\3037800592.py:2: FutureWarning:

Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.



In [65]:
#extract latitude and longitudes from single line geometry
#traffic_signals_exploded['latitude'] = traffic_signals_exploded.apply(lambda x: x.geometry.y, axis=1)
#traffic_signals_exploded['longitude'] = traffic_signals_exploded.apply(lambda x: x.geometry.x, axis=1)

In [66]:
#combining latitude and longitudes to form coordinates column
#traffic_signals_exploded['coordinates'] = traffic_signals_exploded.apply(lambda x: (x.latitude, x.longitude), axis=1)

In [67]:
#create hex cells for traffic

#create coordinate column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded['latitude'].astype('str') + ',' + pedestrian_crossover_exploded['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
#def get_hex_cell(coordinates):
    #a = float(coordinates.split(',')[0])
    #b = float(coordinates.split(',')[1])
    a = coordinates[0]
    b = coordinates[1]
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
#traffic_signals_exploded['hex_cell'] = traffic_signals_exploded['coordinates'].apply(get_hex_cell)

In [ ]:
#traffic_signals_exploded.set_index('id',inplace=True)

In [22]:
#adding flag for traffic signal = 'Yes'
traffic_signals['traffic_signal'] = 'Yes'

In [23]:
#adding prefix before column names
traffic_signals.columns = ['traffic_signal_' + s if s != 'geometry' else s for s in traffic_signals.columns]

In [24]:
traffic_signals.head()

,traffic_signal_main_street,traffic_signal_side1_street,traffic_signal_signalsystem,geometry,traffic_signal_traffic_signal
id,,,,,
1,JARVIS ST,FRONT ST E,TransSuite,MULTIPOINT (-79.37146 43.64945),Yes
2,KING ST E,JARVIS ST,TransSuite,MULTIPOINT (-79.37191 43.65046),Yes
3,JARVIS ST,ADELAIDE ST E,TransSuite,MULTIPOINT (-79.37237 43.65153),Yes
4,JARVIS ST,RICHMOND ST E,TransSuite,MULTIPOINT (-79.37283 43.65270),Yes
5,JARVIS ST,QUEEN ST E,TransSuite,MULTIPOINT (-79.37325 43.65373),Yes


In [27]:
traffic_signals.to_file('traffic_signals_processed.geojson')